
# Beautified reference code

The following cells recreate the main analyses with clearer variable names and structured steps. Each section includes a short explanation of what it accomplishes.



## 1. Load and prepare the master table

This cell loads the Excel workbook, trims column names, joins the four sheets, and adds a binary flag indicating whether each application was approved.


In [ ]:

from pathlib import Path
import numpy as np
import pandas as pd
from IPython.display import display

# Centralized workbook path so it is easy to update in one place
WORKBOOK_PATH = Path('/Users/diego/Documents/vsCode/proyecto_amex_kaggle/sql/Business Case_Business Analyst_REQ_25020234 copy.xlsx')


def load_master_table(workbook_path: Path) -> pd.DataFrame:
    """Load and merge the application, executive, decision, and profile sheets."""
    workbook = pd.ExcelFile(workbook_path)

    applications = pd.read_excel(workbook, 'APLICACION').rename(columns=str.strip)
    executives = pd.read_excel(workbook, 'EJECUTIVO').rename(columns=str.strip)
    decisions = pd.read_excel(workbook, 'DECISION').rename(columns=str.strip)
    profiles = pd.read_excel(workbook, 'PERFIL').rename(columns=str.strip)

    merged_data = (
        applications
        .merge(executives, on='EXECUTIVE_ID', how='left')
        .merge(decisions, on='PROSPECT_ID', how='left')
        .merge(profiles, on='PROSPECT_ID', how='left')
    )

    merged_data['approval_flag'] = np.where(merged_data['DECISION'] == 'APPROVED', 1, 0)
    return merged_data


master_table = load_master_table(WORKBOOK_PATH)
print(f"Loaded {len(master_table):,} rows from the workbook into master_table")



## 2. Group and product performance overview

This block focuses on products 50 and 80, then summarizes volume, approval rate, and the average profile of approved customers by group and product.


In [ ]:

# Keep only the products of interest for the analysis
focus_products = master_table[master_table['PRODUCT_CODE'].isin([50, 80])].copy()

# Volume and approval rate by group and product
volume_and_rate = (
    focus_products
    .groupby(['GROUP', 'PRODUCT_CODE'], as_index=False)
    .agg(
        request_count=('PROSPECT_ID', 'count'),
        approvals=('approval_flag', 'sum')
    )
)

volume_and_rate['approval_rate'] = np.where(
    volume_and_rate['request_count'] > 0,
    volume_and_rate['approvals'] / volume_and_rate['request_count'],
    np.nan,
)

# Profile of approved customers only
approved_customers = focus_products[focus_products['approval_flag'] == 1]
profile_by_group_product = (
    approved_customers
    .groupby(['GROUP', 'PRODUCT_CODE'], as_index=False)
    .agg(
        average_income=('INCOME', 'mean'),
        average_cb_score=('CB_SCORE', 'mean'),
        average_adss_score=('ADSS_SCORE', 'mean'),
    )
)

# Combine counts, rates, and profiles into one table
product_group_summary = volume_and_rate.merge(
    profile_by_group_product,
    on=['GROUP', 'PRODUCT_CODE'],
    how='left',
)

print("Group x Product summary (head):")
display(product_group_summary.head())



## 3. Executive view for GRUPO 02 (including executives with zero requests)

The next cell builds a complete grid of GRUPO 02 executives against products 50 and 80, fills in zero volume where needed, and keeps approval rates as missing when no data exists.


In [ ]:

# Filter the master table to the target group
grupo02_data = master_table[master_table['GROUP'] == 'GRUPO 02'].copy()

# Aggregate only where requests exist
grupo02_metrics = (
    grupo02_data[grupo02_data['PRODUCT_CODE'].isin([50, 80])]
    .groupby(['EXECUTIVE_ID', 'PRODUCT_CODE'], as_index=False)
    .agg(
        request_count=('PROSPECT_ID', 'count'),
        approvals=('approval_flag', 'sum'),
    )
)

# Build the complete grid of executives and products to keep zero-volume rows
executives_list = grupo02_data[['EXECUTIVE_ID']].drop_duplicates()
products_list = pd.DataFrame({'PRODUCT_CODE': [50, 80]})

executive_product_grid = (
    executives_list.assign(key=1)
    .merge(products_list.assign(key=1), on='key')
    .drop(columns='key')
)

grupo02_full_view = executive_product_grid.merge(
    grupo02_metrics,
    on=['EXECUTIVE_ID', 'PRODUCT_CODE'],
    how='left',
)

grupo02_full_view['request_count'] = grupo02_full_view['request_count'].fillna(0).astype(int)
grupo02_full_view['approvals'] = grupo02_full_view['approvals'].fillna(0).astype(int)
grupo02_full_view['approval_rate'] = np.where(
    grupo02_full_view['request_count'] > 0,
    grupo02_full_view['approvals'] / grupo02_full_view['request_count'],
    np.nan,
)

print("GRUPO 02 executive performance (head):")
display(grupo02_full_view.head())



## 4. Low credit score outliers for products 50 and 80

This cell flags applications for products 50 and 80 with a bureau score below 400 and summarizes how many questionable requests each group and executive submitted.


In [ ]:

low_score_outliers = focus_products[focus_products['CB_SCORE'] < 400].copy()

low_score_summary = (
    low_score_outliers
    .groupby(['GROUP', 'EXECUTIVE_ID', 'PRODUCT_CODE'], as_index=False)
    .agg(
        low_score_requests=('PROSPECT_ID', 'count'),
        min_cb_score=('CB_SCORE', 'min'),
        max_cb_score=('CB_SCORE', 'max'),
        approvals_in_low_score=('approval_flag', 'sum'),
    )
    .sort_values('low_score_requests', ascending=False)
)

print("Low credit score outliers (head):")
display(low_score_summary.head())



## 5. Clean customer profile for approved deals

Here we keep only approved applications for products 50 and 80 with a bureau score of at least 400. The result shows the typical income and risk metrics for clean approvals.


In [ ]:

approved_clean = focus_products[
    (focus_products['approval_flag'] == 1)
    & (focus_products['CB_SCORE'] >= 400)
]

clean_product_profile = (
    approved_clean
    .groupby('PRODUCT_CODE', as_index=False)
    .agg(
        approved_customers=('PROSPECT_ID', 'count'),
        median_income=('INCOME', 'median'),
        average_income=('INCOME', 'mean'),
        average_cb_score=('CB_SCORE', 'mean'),
        minimum_cb_score=('CB_SCORE', 'min'),
        maximum_cb_score=('CB_SCORE', 'max'),
        average_adss_score=('ADSS_SCORE', 'mean'),
    )
    .round(2)
)

print("Clean approved customer profile by product:")
display(clean_product_profile)



## 6. Point-of-sale mix and configuration

The final chunk summarizes request and approval volume by point of sale (POS) for products 50 and 80, computes share within each POS, and classifies whether a POS sells only one product or both.


In [ ]:

# Aggregate requests and approvals by POS and product
pos_product_metrics = (
    focus_products
    .groupby(['POINT_OF_SALES_ID', 'PRODUCT_CODE'], as_index=False)
    .agg(
        requests=('PROSPECT_ID', 'count'),
        approvals=('approval_flag', 'sum'),
    )
)

# Wide format for easier share calculations
pos_product_wide = (
    pos_product_metrics
    .pivot(index='POINT_OF_SALES_ID', columns='PRODUCT_CODE', values='requests')
    .fillna(0)
    .rename(columns={50: 'requests_product_50', 80: 'requests_product_80'})
    .reset_index()
)

# Total requests per POS to compute mix
pos_totals = (
    focus_products
    .groupby('POINT_OF_SALES_ID', as_index=False)
    .agg(total_requests=('PROSPECT_ID', 'count'))
)

pos_summary = pos_product_wide.merge(pos_totals, on='POINT_OF_SALES_ID', how='left')

pos_summary['share_product_50'] = np.where(
    pos_summary['total_requests'] > 0,
    pos_summary['requests_product_50'] / pos_summary['total_requests'],
    np.nan,
)
pos_summary['share_product_80'] = np.where(
    pos_summary['total_requests'] > 0,
    pos_summary['requests_product_80'] / pos_summary['total_requests'],
    np.nan,
)

# Classify POS configuration based on which products they sell

def classify_pos_configuration(row: pd.Series) -> str:
    sells_50 = row['requests_product_50'] > 0
    sells_80 = row['requests_product_80'] > 0
    if sells_50 and sells_80:
        return 'both_products'
    if sells_50:
        return 'only_product_50'
    if sells_80:
        return 'only_product_80'
    return 'no_product_50_or_80'

pos_summary['pos_configuration'] = pos_summary.apply(classify_pos_configuration, axis=1)

# Approval-weighted view by configuration
pos_product_approvals = (
    focus_products
    .groupby(['POINT_OF_SALES_ID', 'PRODUCT_CODE'], as_index=False)
    .agg(
        approvals=('approval_flag', 'sum'),
        requests=('PROSPECT_ID', 'count'),
    )
)

config_share = (
    pos_product_approvals
    .pivot_table(
        index='POINT_OF_SALES_ID',
        columns='PRODUCT_CODE',
        values=['approvals', 'requests'],
        aggfunc='sum',
    )
    .fillna(0)
)

config_share.columns = [f"{metric}_product_{int(product)}" for metric, product in config_share.columns]
config_share = config_share.reset_index()

config_with_labels = config_share.merge(pos_summary[['POINT_OF_SALES_ID', 'pos_configuration']], on='POINT_OF_SALES_ID', how='left')

configuration_summary = (
    config_with_labels
    .groupby('pos_configuration', as_index=False)
    .agg(
        pos_count=('POINT_OF_SALES_ID', 'nunique'),
        requests_product_50=('requests_product_50', 'sum'),
        requests_product_80=('requests_product_80', 'sum'),
        approvals_product_50=('approvals_product_50', 'sum'),
        approvals_product_80=('approvals_product_80', 'sum'),
    )
)

configuration_summary['approval_rate_product_50'] = np.where(
    configuration_summary['requests_product_50'] > 0,
    configuration_summary['approvals_product_50'] / configuration_summary['requests_product_50'],
    np.nan,
)
configuration_summary['approval_rate_product_80'] = np.where(
    configuration_summary['requests_product_80'] > 0,
    configuration_summary['approvals_product_80'] / configuration_summary['requests_product_80'],
    np.nan,
)

print("POS-level summary (head):")
display(pos_summary.head())
print("
POS configuration summary:")
display(configuration_summary)


## Beautified code (appended)

The following cells restate the workflow with clearer variable names, explicit helper functions, and brief explanations of each step.

### Imports and workbook path

This cell gathers all library imports in one place and defines the workbook path constant.

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
from IPython.display import display

WORKBOOK_PATH = Path('/Users/diego/Documents/vsCode/proyecto_amex_kaggle/sql/Business Case_Business Analyst_REQ_25020234 copy.xlsx')

### Load workbook sheets into DataFrames

This helper reads the Excel workbook once and trims column names so downstream joins stay consistent.

In [ ]:
def load_workbook_tables(excel_path: Path) -> dict[str, pd.DataFrame]:
    "Read each relevant sheet into a tidy DataFrame."
    workbook = pd.ExcelFile(excel_path)
    sheet_map = {
        'applications': 'APLICACION',
        'executives': 'EJECUTIVO',
        'decisions': 'DECISION',
        'profiles': 'PERFIL',
    }
    return {alias: pd.read_excel(workbook, sheet).rename(columns=str.strip) for alias, sheet in sheet_map.items()}

### Merge sheets into a master table

This function combines the four sheets, adds a binary approval flag, and reports the final shape.

In [ ]:
def build_master_table(tables: dict[str, pd.DataFrame]) -> pd.DataFrame:
    "Merge application, executive, decision, and profile sheets into one table."
    merged = (
        tables['applications']
        .merge(tables['executives'], on='EXECUTIVE_ID', how='left')
        .merge(tables['decisions'], on='PROSPECT_ID', how='left')
        .merge(tables['profiles'], on='PROSPECT_ID', how='left')
    )
    merged['is_approved'] = (merged['DECISION'] == 'APPROVED').astype(int)
    return merged

workbook_tables = load_workbook_tables(WORKBOOK_PATH)
master_table = build_master_table(workbook_tables)
print(f"Loaded master table with {master_table.shape[0]:,} rows and {master_table.shape[1]} columns.")

### Summarize volume and approval rate by group and product

This chunk filters to products 50 and 80, then calculates request volume, approvals, approval rate, and profile averages by group.

In [ ]:
TARGET_PRODUCTS = [50, 80]
filtered_products = master_table[master_table['PRODUCT_CODE'].isin(TARGET_PRODUCTS)].copy()

group_product_volume = (
    filtered_products
    .groupby(['GROUP', 'PRODUCT_CODE'], as_index=False)
    .agg(requests=('PROSPECT_ID', 'count'), approvals=('is_approved', 'sum'))
)
group_product_volume['approval_rate'] = np.where(
    group_product_volume['requests'] > 0,
    group_product_volume['approvals'] / group_product_volume['requests'],
    np.nan,
)

approved_profiles = filtered_products[filtered_products['is_approved'] == 1]
group_product_profile = (
    approved_profiles
    .groupby(['GROUP', 'PRODUCT_CODE'], as_index=False)
    .agg(
        average_income=('INCOME', 'mean'),
        average_cb_score=('CB_SCORE', 'mean'),
        average_adss_score=('ADSS_SCORE', 'mean'),
    )
)

group_product_overview = group_product_volume.merge(
    group_product_profile, on=['GROUP', 'PRODUCT_CODE'], how='left'
)
print("Group and product overview (head):")
display(group_product_overview.head())

### Executive-level grid for GRUPO 02

This cell builds a complete matrix of GRUPO 02 executives against products 50 and 80, filling missing combinations with zeros.

In [ ]:
group02_records = filtered_products[filtered_products['GROUP'] == 'GRUPO 02'].copy()

executive_roster = (
    master_table[master_table['GROUP'] == 'GRUPO 02'][['EXECUTIVE_ID', 'EXECUTIVE_NAME']]
    .drop_duplicates()
)
product_options = pd.DataFrame({'PRODUCT_CODE': TARGET_PRODUCTS})

exec_product_grid = (
    executive_roster.assign(key=1)
    .merge(product_options.assign(key=1), on='key')
    .drop(columns='key')
)

executive_summary = (
    group02_records
    .groupby(['EXECUTIVE_ID', 'EXECUTIVE_NAME', 'PRODUCT_CODE'], as_index=False)
    .agg(requests=('PROSPECT_ID', 'count'), approvals=('is_approved', 'sum'))
)
executive_summary['approval_rate'] = np.where(
    executive_summary['requests'] > 0,
    executive_summary['approvals'] / executive_summary['requests'],
    np.nan,
)

executive_grid = exec_product_grid.merge(
    executive_summary, on=['EXECUTIVE_ID', 'EXECUTIVE_NAME', 'PRODUCT_CODE'], how='left'
)
executive_grid[['requests', 'approvals']] = executive_grid[['requests', 'approvals']].fillna(0).astype(int)

print("Executive grid for GRUPO 02 (head):")
display(executive_grid.head())

### Branch-level request and approval summary for GRUPO 02

This block aggregates GRUPO 02 results by branch and product, computing both counts and approval rates.

In [ ]:
branch_summary = (
    group02_records
    .groupby(['BRANCH', 'PRODUCT_CODE'], as_index=False)
    .agg(requests=('PROSPECT_ID', 'count'), approvals=('is_approved', 'sum'))
)
branch_summary['approval_rate'] = np.where(
    branch_summary['requests'] > 0,
    branch_summary['approvals'] / branch_summary['requests'],
    np.nan,
)

print("Branch summary for GRUPO 02:")
display(branch_summary.head())

### Profile of approved customers with CB score ≥ 400

This step filters to approved customers above the credit bureau score threshold and summarizes their profile by product.

In [ ]:
approved_high_cb = filtered_products[(filtered_products['is_approved'] == 1) & (filtered_products['CB_SCORE'] >= 400)]

approved_profile = (
    approved_high_cb
    .groupby('PRODUCT_CODE', as_index=False)
    .agg(
        approved_customers=('PROSPECT_ID', 'count'),
        median_income=('INCOME', 'median'),
        average_income=('INCOME', 'mean'),
        average_cb_score=('CB_SCORE', 'mean'),
        minimum_cb_score=('CB_SCORE', 'min'),
        maximum_cb_score=('CB_SCORE', 'max'),
        average_adss_score=('ADSS_SCORE', 'mean'),
    )
    .round(2)
)

print("Approved profile with CB score ≥ 400:")
display(approved_profile)

### Point-of-sale mix for products 50 and 80

The final cell summarizes how each point of sale sells products 50 and 80, including request mix, approvals, and configuration labels.

In [ ]:
pos_metrics = (
    filtered_products
    .groupby(['POINT_OF_SALES_ID', 'PRODUCT_CODE'], as_index=False)
    .agg(requests=('PROSPECT_ID', 'count'), approvals=('is_approved', 'sum'))
)

pos_totals = (
    filtered_products
    .groupby('POINT_OF_SALES_ID', as_index=False)
    .agg(total_requests=('PROSPECT_ID', 'count'))
)

pos_pivot = (
    pos_metrics
    .pivot(index='POINT_OF_SALES_ID', columns='PRODUCT_CODE', values='requests')
    .fillna(0)
    .rename(columns={50: 'requests_product_50', 80: 'requests_product_80'})
    .reset_index()
)

pos_mix = pos_pivot.merge(pos_totals, on='POINT_OF_SALES_ID', how='left')
pos_mix['share_product_50'] = np.where(
    pos_mix['total_requests'] > 0,
    pos_mix['requests_product_50'] / pos_mix['total_requests'],
    np.nan,
)
pos_mix['share_product_80'] = np.where(
    pos_mix['total_requests'] > 0,
    pos_mix['requests_product_80'] / pos_mix['total_requests'],
    np.nan,
)

pos_configuration = []
for _, row in pos_mix.iterrows():
    sells_50 = row['requests_product_50'] > 0
    sells_80 = row['requests_product_80'] > 0
    if sells_50 and sells_80:
        pos_configuration.append('Sells both 50 and 80')
    elif sells_50:
        pos_configuration.append('Only sells 50')
    elif sells_80:
        pos_configuration.append('Only sells 80')
    else:
        pos_configuration.append('No requests for 50 or 80')
pos_mix['pos_configuration'] = pos_configuration

print("Point-of-sale mix summary (head):")
display(pos_mix.head())